General imports

In [ ]:
import numpy as np
import simcalc

In [ ]:
%load_ext autoreload
%autoreload 2

Plotting

In [ ]:
import holoviews as hv
hv.extension('bokeh', 'matplotlib')
%opts Raster (cmap='Greys_r') Image (cmap='Greys_r')

Generate basic spikes and calcium traces

In [ ]:
#gcamp6f stats
taur = 0.1
taud = 0.76
A = 1
p = [0.85, -0.006]

# define other inputs
nCells = 5
h = 50 # height and width of field of view
T = 7.5 # final time
dt= 0.01 # timestep
cycles = 2 # how many stimulus cycles
rates = np.linspace(0,1,nCells)
rates[0] = 0.2
rates[np.arange(1,nCells,3)] = 0.5
rates[np.arange(2,nCells,3)] = 0.5

# run model
model = simcalc.SimCalc(h, nCells, T, cycles, dt)
model.gen_calcium(taur,taud,A,p, rates=rates)

In [ ]:
%%opts Curve [width=900] Curve.spikes (color='k') Curve.F (color='g')
sp = model.sp # spikes
c = model.c # inherent calcium level without nonlinearities
F = model.F # calcium levels with measurement nonlinearities taken into account
times = np.arange(F.shape[1])*dt # time points

# plot spikes and calcium traces
calc_fig = hv.Overlay()
offset=6
for t in range(sp.shape[0]): # loop over trials
    calc_fig *= hv.Curve(zip(times, sp[t, :]*3+offset*t), group='spikes', label='spikes')
    calc_fig *= hv.Curve(zip(times, F[t, :]+offset*t), group='F', label='calcium measurement')
calc_fig

Generate neuropil signal and spatial footprint

In [ ]:
model.gen_npil()

Generate spatial kernels for the cells

In [ ]:
model.gen_spat_kernels()

Generate video data

In [ ]:
model.gen_data(p)

Final video

In [ ]:
%%output holomap='scrubber'
subt = 10 # downsampling
startT = 0
endT = model.F.shape[1]
frames = {f: hv.Image(model.video[int(f-subt/2):int(f+subt/2),:,:].mean(axis=0)) for f in np.arange(startT+subt/2,endT,subt)}
hv.HoloMap(frames)